In [1]:
%matplotlib inline  
import pandas as pd

In [2]:
# get the data from csv file.
df = pd.read_csv('train.csv', index_col=0)
X_test = pd.read_csv('test.csv', index_col=0)

In [3]:
# dicard unnecessary data
df = df[['Survived', 'Pclass', 'Sex', 'Age', 'SibSp', 'Parch']]
X_test = X_test[['Pclass', 'Sex', 'Age', 'SibSp', 'Parch']]

In [4]:
# divide age into two categories, child and adult
df['Age'] = pd.cut(df['Age'], [0, 8, df['Age'].max()], labels=['child', 'adult'])
X_test['Age'] = pd.cut(X_test['Age'], [0, 8, X_test['Age'].max()], labels=['child', 'adult'])

X_test['Family'] = X_test['SibSp'] + X_test['Parch']
X_test = X_test.drop(['SibSp', 'Parch'], 1)

In [5]:
# create a new feature 'family'
df['Family'] = df['SibSp'] + df['Parch']
df = df.drop(['SibSp', 'Parch'], 1)

In [6]:
# change categories to numbers
df['Sex'] = df['Sex'].apply(lambda x: 1 if x == "male" else 0)
df['Family'] = df['Family'].apply(lambda x: 4 if x > 4 else x)
df['Age'] = df['Age'].apply(lambda x: 1 if x == "adult" else 0)

X_test['Sex'] = X_test['Sex'].apply(lambda x: 1 if x == "male" else 0)
X_test['Family'] = X_test['Family'].apply(lambda x: 4 if x > 4 else x)
X_test['Age'] = X_test['Age'].apply(lambda x: 1 if x == "adult" else 0)

In [7]:
# obtain X_train and y_train from the dataframe.
X_train = df.drop('Survived', axis = 1) 
y_train = df['Survived']

In [8]:
# fill incomplete data in X_train
from sklearn.preprocessing import Imputer
imputer = Imputer(strategy='median', axis=1)
imputer = imputer.fit(X_train)
X_train = imputer.transform(X_train)

# fill incomplete data in X_test

imputer = Imputer(strategy='median', axis=1)
imputer = imputer.fit(X_test)
X_test = imputer.transform(X_test)

In [9]:
from sklearn.preprocessing import PolynomialFeatures
poly = PolynomialFeatures(2)
X_train = poly.fit_transform(X_train)  
X_test = poly.fit_transform(X_test)  

In [10]:
# fit the model
from sklearn.linear_model import LogisticRegression
clf = LogisticRegression()
clf = clf.fit(X_train, y_train)

In [14]:
print("Training Accuracy:", clf.score(X_train, y_train))

Training Accuracy: 0.824915824916


In [12]:
# get the result
import numpy as np
result = pd.DataFrame(data=clf.predict(X_test), index = np.arange(892, 1310), columns=['Survived'])

In [13]:
# save the result to csv file
result.index.name = 'PassengerId'
result.to_csv('result.csv')